In [1]:
from functools import reduce

In [2]:
def bruteforce_inv(a, p):
    for b in range(p):
        if (a * b) % p == 1: return b
    return None

def discr_pow(a, n, p):
    if n == 0: return 1
    if n % 2 == 0:
        half = discr_pow(a, n / 2, p) % p
        return (half ** 2) % p;
    return (a * discr_pow(a, n-1, p)) % p

def discr_div(a, b, p):
    binv = bruteforce_inv(b, p)
    if binv == None: return None
    return (a * binv) % p

def bruteforce_discr_log(b, y, p):
# log_{b}(y) (mod p) = n = (?)
    pw = 1
    for n in range(p):
        if pw == y: return n
        pw = (pw * b) % p
    return None

def chinese_rem(r: list, p: list):
    assert(len(r) == len(p))
    L = len(p)
    P = reduce((lambda a, res: a * res), p);
    M = [P // p[i] for i in range(L)]
    Minv = [bruteforce_inv(M[i], p[i]) for i in range(L)]
    prods = map(lambda a: a[0] * a[1] * a[2], zip(r, M, Minv))
    return reduce(lambda a, res: (a + res) % P, prods)

In [3]:
def SPH(b: int, y: int, p: list, pows: list):
# Silver-Pollig-Hellman algorithm
# b -- base of log
# y -- argument
# sum(p .^ pows) + 1 = q  --  mod

    assert(len(p) == len(pows));
    L = len(p);
    q = 1 + reduce((lambda a, res: a * res), 
                   [p[i] ** pows[i] for i in range(L)])
    
    # compute primitive roots
    unity = []
    for i in range(L):
        mul = discr_pow(b, (q - 1) // p[i], q)
        cur = 1
        lst = []
        for j in range(p[i]):
            lst.append(cur)
            cur = (cur * mul) % q
        unity.append(lst)
    
    # compute x mod p[i] ^ pow[i]
    X = []
    for i in range(L):
        y1 = y
        res = 0
        for j in range(pows[i]):
            pw  =  discr_pow(y1, (q - 1) // (p[i]**(j + 1)), q)
            upd =  unity[i].index(pw) * p[i]**j
            y1 = discr_div(y1, b ** upd, q)
            res += upd
        X.append(res);
    
    # use chinese remainders theorem
    pws = list(map((lambda a: a[0] ** a[1]), zip(p, pows)));
    return chinese_rem(X, pws)


In [177]:
log = SPH(2, 153, [2, 3, 5], [2, 2, 1])
print("log = %d" % log)
print("2 ^ log = %d" % discr_pow(2, log, 2**2 * 3**2 * 5**1 + 1))

log = 107
2 ^ log = 153


In [178]:
log = SPH(2, 28, [2, 3], [2, 2])
print("log = %d" % log)
print("2 ^ log = %d" % discr_pow(2, log, 2**2 * 3**2 + 1))

log = 34
2 ^ log = 28


In [179]:
log = SPH(5, 23, [2, 3], [5, 1])
print("log = %d" % log)
print("5 ^ log = %d" % discr_pow(5, log, 2**5 * 3**1 + 1))

log = 77
5 ^ log = 23


In [5]:
p1 = 2; n1 = 2;
p2 = 3; n2 = 2;
p3 = 5; n3 = 1;
q  = 181;
y  = 153;
b  = 2;

In [6]:
# p1 = 2
R1 = []
for j in range(p1):
    R1.append((b ** (j * (q - 1) / p1)) % q);
    print("%d -> %d" % (j, R1[j]) )

0 -> 1
1 -> 180


In [7]:
# p2 = 3
R2 = []
for j in range(p2):
    R2.append((b ** (j * (q - 1) / p2)) % q)
    print("%d -> %d" % (j, R2[j]) )

0 -> 1
1 -> 48
2 -> 132


In [8]:
# p3 = 5
R3 = []
for j in range(p3):
    R3.append((b ** (j * (q - 1) / p3)) % q)
    print("%d -> %d" % (j, R3[j]) )

0 -> 1
1 -> 59
2 -> 42
3 -> 125
4 -> 135


In [9]:
# x mod 2^2
X1 = []

tmp = discr_pow(y, (q - 1) / p1, q)
X1.append(R1.index(tmp))
print(tmp)
print("%d * %d^0" % (X1[0], p1))

y1 = discr_div(y, b ** (X1[0] * p1 ** 0), q);
tmp = discr_pow(y1, (q - 1) / (p1**2), q)
X1.append(R1.index(tmp))
print(tmp)
print("%d * %d^1" % (X1[1], p1))

X1 = (X1[0] + p1 * X1[1]) % q
print("final: %d" % X1)

180
1 * 2^0
180
1 * 2^1
final: 3


In [20]:
# x mod 3^2
X2 = []

tmp = discr_pow(y, (q - 1) / p2, q)
X2.append(R2.index(tmp))
print(tmp)
print("%d * %d^0" % (X2[0], p2))

y1 = discr_div(y, b ** (X2[0] * p2 ** 0), q);
print(y1)
tmp = discr_pow(y1, (q - 1) / (p2**2), q)
X2.append(R2.index(tmp))

print("%d * %d^1" % (X2[1], p2))

X2 = (X2[0] + p2 * X2[1]) % q
print("final: %d" % X2)

132
2 * 3^0
174
2 * 3^1
final: 8


In [11]:
# x mod 5^1
X3 = []

tmp = discr_pow(y, (q - 1) / p3, q)
X3.append(R3.index(tmp))
print(tmp)
print("%d * %d^0" % (X3[0], p3))

X3 = (X3[0]) % q
print("final: %d" % X3)

42
2 * 5^0
final: 2


In [187]:
# compose
chinese_rem([X1, X2, X3], [p1**2, p2**2, p3])

107

In [188]:
# check
bruteforce_discr_log(b, y, q)

107

In [37]:
discr_div(17, 8, 163)

104

In [40]:
discr_pow(104, 6, 163)

1